In [1]:
import csv
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import save_npz
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
CUDA_VISIBLE_DEVICES=""
tf.config.run_functions_eagerly(True)


csv.field_size_limit(100000000)

2023-11-23 02:32:18.511705: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-23 02:32:18.511735: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-23 02:32:18.512777: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-23 02:32:18.518896: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-23 02:32:19.154469: W tensorflow/comp

131072

In [2]:
def parse_csv(filename):
    data = []
    with open(filename, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        next(reader)  # Skip the header row
        for row in reader:
            email_text = row[1].replace('""', '"')  # Replace double double quotes with a single quote
            email_type = 0 if row[2]=="Safe Email" else 1
            data.append({'Email Text': email_text, 'Email Type': email_type})

    data = pd.DataFrame(data)
    return data

# Example usage
filename = 'data/Phishing_Email.csv'
parsed_data = parse_csv(filename)

print(len(parsed_data))


18650


In [8]:
parsed_data.iloc[0]

Email Text    re : 6 . 1100 , disc : uniformitarianism , re ...
Email Type                                                    0
Name: 0, dtype: object

In [29]:
X = parsed_data['Email Text'].astype(str)
y = parsed_data['Email Type'].to_numpy()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [30]:
# Save the TF-IDF transformed data to a file
save_npz('trained_weights/X_train_tfidf.npz', X_train_tfidf)
save_npz('trained_weights/X_test_tfidf.npz', X_test_tfidf)
np.save('trained_weights/y_train.npy', y_train)
np.save('trained_weights/y_test.npy', y_test)

In [4]:
from scipy.sparse import load_npz

loaded_X_train_tfidf = load_npz('trained_weights/X_train_tfidf.npz')
loaded_X_test_tfidf = load_npz('trained_weights/X_test_tfidf.npz')

In [4]:
feature_names = tfidf_vectorizer.get_feature_names_out()
avg_tfidf_values = X_train_tfidf.mean(axis=0).A1
sorted_features = sorted(zip(feature_names, avg_tfidf_values), key=lambda x: x[1], reverse=True)

# Print the top N features and their average TF-IDF values
top_n = 10
print(f"\nTop {top_n} features based on average TF-IDF values:")
for feature, avg_tfidf in sorted_features[:top_n]:
    print(f"{feature}: {avg_tfidf}")


Top 10 features based on average TF-IDF values:
com: 0.030139118236201953
http: 0.028007689681069547
enron: 0.021340140625526867
www: 0.019524336890116338
ect: 0.01662853034408202
list: 0.016306305098285113
linux: 0.015944428273739174
click: 0.01570476661884602
email: 0.015527566554499042
net: 0.014617029332719038


In [11]:
print(X_train_tfidf.shape)
print(len(y_train))
print(X_test_tfidf.shape)
print(len(y_test))
print(y_train[0])
print(y_train[2])

(14920, 300)
14920
(3730, 300)
3730
0
0


In [20]:
X_train_tfidf_train = X_train_tfidf.toarray()
#X_train_tfidf_train = X_train_tfidf.toarray().reshape((X_train_tfidf.shape[0], X_train_tfidf.shape[1], 1))
X_test_tfidf_train = X_test_tfidf.toarray()
#X_test_tfidf_train = X_test_tfidf.toarray().reshape((X_test_tfidf.shape[0], X_test_tfidf.shape[1], 1))


print(X_train_tfidf_train.shape)
print(y_train.shape)
print(X_train_tfidf.toarray().shape)

print(type(X_train_tfidf_train))
print(type(y_train))
print(type(y_train[0]))
X_train_tfidf_train.astype('float64')

y_train = y_train.astype('int32')
y_test = y_test.astype('int32')

# Build a fully connected feedforward neural network
model = Sequential()
model.add(Dense(128, input_shape=(X_train_tfidf_train.shape[1],), activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification, adjust for your task

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

# Train the model
model.fit(X_train_tfidf_train, y_train, epochs=10, batch_size=64, validation_data=(X_test_tfidf_train, y_test))

(14920, 300)
(14920,)
(14920, 300)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.int32'>
Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 128)               38528     
                                                                 
 dense_41 (Dense)            (None, 128)               16512     
                                                                 
 dense_42 (Dense)            (None, 128)               16512     
                                                                 
 dense_43 (Dense)            (None, 1)                 129       
                                                                 
Total params: 71681 (280.00 KB)
Trainable params: 71681 (280.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/10


2023-11-23 02:46:10.655099: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: JIT compilation failed.


UnknownError: Exception encountered when calling layer 'dense_43' (type Dense).

{{function_node __wrapped__Sigmoid_device_/job:localhost/replica:0/task:0/device:GPU:0}} JIT compilation failed. [Op:Sigmoid] name: 

Call arguments received by layer 'dense_43' (type Dense):
  • inputs=tf.Tensor(shape=(64, 128), dtype=float32)

In [ ]:
# Build the CNN model
model = models.Sequential()
model.add(layers.Conv1D(32, 3, activation='relu', input_shape=(5000, 1)))
model.add(layers.MaxPooling1D(2))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  # Change to sigmoid for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

# Define a custom callback to print intermediate outputs
class IntermediateOutputCallback(Callback):
    def __init__(self, layer_names):
        super(IntermediateOutputCallback, self).__init__()
        self.layer_names = layer_names

    def on_epoch_end(self, epoch, logs=None):
        intermediate_layer_models = [tf.keras.Model(inputs=model.input, outputs=model.get_layer(name).output) for name in self.layer_names]

        for i, name in enumerate(self.layer_names):
            intermediate_output = intermediate_layer_models[i].predict(X_train_tfidf)
            print(f"Intermediate Output of Layer '{name}' during epoch {epoch + 1}: {intermediate_output}")

# Instantiate the callback with the names of layers for which you want to inspect outputs
callback = IntermediateOutputCallback(layer_names=['conv1d', 'dense36'])

#print(type(X_test_tfidf_train))

# Train the model with the custom callback
model.fit(X_train_tfidf_train, y_train, epochs=5, batch_size=64, validation_split=0.2, callbacks=[callback])

# Train the model
#model.fit(X_train_tfidf_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test_tfidf_train, y_test)
print(f'Test accuracy: {test_acc}')